# Melting Dynamics

This is a notebook to understand the dynamics of crystal melting, in particular how much of the slowdown of the melting can be attributed to the slow dynamics.

In [1]:
# Import required modules
import pandas as pd
import numpy as np
import altair as alt
from ipywidgets import interact, ToggleButtons, IntSlider

## Input data

The data for this notebook is sourced from the calculations of the relaxation times which can be run using the makefile in the home directory of this repository. The command `make relaxations` will compute the relaxation values that are required for this analysis, although if not run previously the command `make dynamics` will also need to be run as I am yet to set up any dependencies in the Makefile.

The calculation of the melting rates is performed in the [Crystal_Clustering](Crystal_Clustering.ipynb) notebook from calculations of crystal fractions from running `make melting`.

In [2]:
df_relax = pd.read_hdf('../data/analysis/dynamics.h5', 'relaxations')
df_relax.reset_index(inplace=True)
df_relax['1/D'] = 1 / df_relax.diffusion_constant
df_relax['temperature'] = df_relax['temperature'].astype(float)
df_relax['pressure'] = df_relax['pressure'].astype(float)

df_melting = pd.read_hdf("../data/analysis/melting.h5", "rates")
df_melting['temperature'] = df_melting['temperature'].astype(float)
df_melting['pressure'] = df_melting['pressure'].astype(float)
df_melting['inv_melting_rate'] = np.abs(1/df_melting['melting_rate'].values)
df_melting.set_index(['temperature', 'pressure'], inplace=True)

KeyError: 'No object named rates in the file'

In [4]:
df_relax

,index,temperature,pressure,tau_D1_mean,tau_D1_hmean,tau_D04_mean,tau_D04_hmean,tau_DL04_mean,tau_DL04_hmean,tau_T2_mean,...,tau_T4_mean,tau_T4_hmean,max_alpha_time,tau_F,max_gamma_time,diffusion_constant,tau_R1,tau_R2,tau_S,1/D
0,0,0.30,1.0,2.096300e+08,2.094678e+08,773858.221713,697386.225897,1.643502e+08,1.641808e+08,3.032743e+08,...,2.079345e+08,2.079128e+08,11000000.0,1.090068e+08,680000.0,1.222011e-09,5.759919e+08,2.788471e+08,2.894286e+07,8.183234e+08
1,1,0.32,1.0,5.441240e+07,5.322547e+07,40807.736539,34459.077345,4.378301e+07,4.279387e+07,1.998029e+08,...,1.059845e+08,1.057854e+08,970000.0,1.610489e+07,150000.0,9.922463e-09,1.370868e+08,6.937867e+07,2.072646e+06,1.007814e+08
2,2,0.34,1.0,1.373157e+07,1.358263e+07,13021.817967,10820.108444,1.129312e+07,1.116001e+07,5.821221e+07,...,2.710314e+07,2.707248e+07,3000000.0,6.261412e+06,200000.0,6.673315e-08,3.421246e+07,1.711150e+07,1.504499e+06,1.498506e+07
3,3,0.36,1.0,2.697862e+06,2.644909e+06,2665.836095,2437.331581,2.273599e+06,2.222217e+06,1.524624e+07,...,6.510243e+06,6.496547e+06,440000.0,1.409174e+06,57000.0,3.483607e-07,1.042827e+07,5.039390e+06,2.736641e+05,2.870588e+06
4,4,0.38,1.0,5.184879e+05,4.733471e+05,1260.868481,1222.667550,4.299859e+05,3.929758e+05,5.881165e+06,...,2.067317e+06,2.053120e+06,110000.0,2.347205e+05,20000.0,1.154805e-06,3.712776e+06,1.438170e+06,3.956501e+04,8.659473e+05
5,5,0.40,1.0,1.593024e+05,1.480187e+05,877.708343,866.832775,1.348187e+05,1.244328e+05,2.470475e+06,...,8.288294e+05,8.249856e+05,43000.0,6.414946e+04,9700.0,3.360863e-06,1.590870e+06,5.722708e+05,1.045249e+04,2.975426e+05
6,6,0.42,1.0,6.295469e+04,5.991019e+04,704.072107,698.840601,5.297815e+04,5.031302e+04,1.385412e+06,...,4.141489e+05,4.114670e+05,28000.0,2.779647e+04,7100.0,7.470865e-06,8.903372e+05,2.895136e+05,3.816042e+03,1.338533e+05
7,7,0.44,1.0,2.569273e+04,2.487355e+04,581.350874,578.205265,2.168620e+04,2.092080e+04,7.422459e+05,...,2.067742e+05,2.062928e+05,12000.0,1.023989e+04,3500.0,1.684971e-05,4.841425e+05,1.471765e+05,1.645189e+03,5.934819e+04
8,8,0.46,1.0,1.397627e+04,1.357374e+04,507.505589,505.664019,1.158117e+04,1.119102e+04,4.637367e+05,...,1.262327e+05,1.259203e+05,9100.0,4.981600e+03,3900.0,3.339974e-05,3.052606e+05,8.956094e+04,8.588663e+02,2.994036e+04
9,9,0.50,1.0,6.210122e+03,6.092812e+03,408.371483,407.427364,4.946054e+03,4.833335e+03,2.325409e+05,...,5.990234e+04,5.977442e+04,4800.0,2.163122e+03,1100.0,8.582062e-05,1.538808e+05,4.304783e+04,4.870753e+02,1.165221e+04


## Merge Dataframes

This merges the dataframes containing the data on the timescale of relaxations and the timescale of melting.

In [ ]:
df_timescales = df_relax.merge(df_melting, on=['temperature', 'pressure'])
df_timescales['pressure'] = df_timescales['pressure'].astype(str)

In [ ]:
df_timescales.columns

In [ ]:
df_timescales.pressure.astype('str')

In [ ]:
def create_chart(axis):
    chart = alt.Chart(df_timescales, width=400, height=500).mark_point(size=75).transform_calculate(
        '1/T', '1/datum.temperature'
    ) .encode(
        x=alt.X('1/T:Q', scale=alt.Scale(zero=False)),
        y=alt.Y(axis, type='quantitative', axis=alt.Axis(format='e'), scale=alt.Scale(type='log')),
        color="crys:N",
        tooltip=[alt.Tooltip(f'{axis}:Q', format='.2e')]
    )
    return chart.transform_filter(alt.datum.pressure == '1.0')

In [ ]:
create_chart('inv_melting_rate') + create_chart('tau_T4_mean').mark_point(filled=True)

In [ ]:
df_timescales['tau_T4_melting_rate'] = df_timescales["tau_T4_mean"]*df_timescales["melting_rate"].abs()
df_timescales['timescale_per_event'] = 1/df_timescales['tau_T4_melting_rate']

In [ ]:
c = create_chart("tau_T4_melting_rate")

In [ ]:
c.mark_point(filled=False)

In [ ]:
df_timescales.query('pressure == "1.0"').melting_rate